# Train and generate a Random Forest model


Author: Meylin Herrera \\
mhscience525@gmail.com
MSc_thesis_landslide_Detection-2019 (Deltares-TUDelft)    
Description: to train a Random Forest model for landslides detection. The input data are tables derived from segmentation of optical satellite images (Sentinel-2)

In [ ]:
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import norm
from scipy import stats
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler


 Required directories
    

In [ ]:
#Add your path for:
segmentation_tables_path = '' 
confusion_matrix_output_path = '' 
model_output_path = '' 
correlation_graph_output_path = '' 
feature_importance_output_path = ''

 Read segmentation tables and create data frames

In [ ]:
path = segmentation_tables_path

# datasets
landslides_seg = {}
files = glob.glob('data*.csv')
for filename in os.listdir(path):  
     if filename.endswith('.csv'):
        frame = pd.read_csv(path+filename,index_col=False)
        frame.rename( columns={'Unnamed: 0':'segment'}, inplace=True)  #rename columns   
        landslides_seg[filename[0:12]] = frame   

In [ ]:
landslides_seg.keys()

In [ ]:
# landslides_seg['landslide_0_'].head() #check data structure
#check landslides = 1/non_landslides segments =0
# landslides_seg['landslide_0_'].loc[landslides_seg['landslide_0_']['class']==1] 

## 2. Data cleaning and  landslides features computation 


Eliminate outliers using z-score

In [ ]:
for key in landslides_seg:
    cols = landslides_seg[key][['ndvi','slope_mean','brightness','ndvi_change','ratio_rg_change','ndvi_change']]
    z = np.abs(stats.zscore(cols))
    print ("Maximum Z:", z.max(), key ) # show that outlier have been detected
    landslides_seg[key] = landslides_seg[key][(z <5).all(axis=1)]# observation outside 5 standard deviations is considered as an outlier

Calculate contextual features: landslide diagnostic features relative to the information contained in the image. It is calculated as the difference between the segment (feature value) and the weighted mean of the image.

In [ ]:
def neighbours_relationship(df_train,feature,area):
    for key in df_train:
        #calculate the weighted mean per feature
        weighted_mean = (df_train[key][feature] * df_train[key][area]).sum() /(df_train[key][area].sum())
        mean_all_segments = df_train[key][feature].mean()
        feature_subtraction_weighted = []        

        for i in range (len(df_train[key])):
            # Subtract the mean from each observation and squared it
              mean_weighted_subtraction = (df_train[key][feature].iloc[i] - weighted_mean) 
              mean_subtraction  =  (df_train[key][feature].iloc[i] - mean_all_segments) 
              new_name_feature = feature[0:]+'_var'
              new_name_feature_weighted = feature[0:]+'_deviation'    
              feature_subtraction_weighted.append (mean_weighted_subtraction)

        # Create a new column with the calculated contextual feature
        df_train[key][new_name_feature_weighted] = feature_subtraction_weighted   
     

neighbours_relationship(landslides_seg,'ndvi','area_m2')             
neighbours_relationship(landslides_seg,'ratio_rg_change','area_m2')
neighbours_relationship(landslides_seg,'brightness','area_m2')
neighbours_relationship(landslides_seg,'gndvi','area_m2')
neighbours_relationship(landslides_seg,'ndvi_change','area_m2')
neighbours_relationship(landslides_seg,'brightness_change','area_m2')
neighbours_relationship(landslides_seg,'nd_std','area_m2')

Calculate relative relieve: difference between the highest and lowest points in elevation within the segments

In [ ]:
def relative_relief (df_train,height_min, height_max):
    
     for key in df_train:
            relative_relief_list = []  
            
            for i in range (len(df_train[key])):
                relative_relief = (df_train[key][height_max].iloc[i] - df_train[key][height_min].iloc[i] )
                relative_relief_list.append (relative_relief)
                
            df_train[key]['relative_relief'] = relative_relief_list  

relative_relief (landslides_seg, 'height_min', 'height_max')

Create a unique dataset with all segmented tables

In [ ]:
df_keys = pd.concat(landslides_seg, ignore_index=True)

In [ ]:
#Count the number of landslides segments
df_keys.loc[df_keys.loc[:,'class']==1,:].count() 

In [ ]:
#Count the number of non- landslides segments
df_keys.loc[df_keys.loc[:,'class']==0,:].count() 

## 3. Data visualization

### Data Normalization 

In [ ]:
#Normalize data for visualization purposes

In [ ]:
# #create a new df to normalize the features values
df_norm_data = df_keys.copy()
df_norm_data.columns

In [ ]:
feature_normalization=[]
# df_keys.columns
feature_normalization.append((df_norm_data.columns[2], df_norm_data.columns[3], df_norm_data.columns[4],  
                              df_norm_data.columns[5], df_norm_data.columns[6],df_norm_data.columns[7],
                              df_norm_data.columns[10],df_norm_data.columns[11],df_norm_data.columns[12],
                              df_norm_data.columns[13], df_norm_data.columns[14], df_norm_data.columns[15],  
                              df_norm_data.columns[16], df_norm_data.columns[17],df_norm_data.columns[18],
                              df_norm_data.columns[19],df_norm_data.columns[20],df_norm_data.columns[21],
                              df_norm_data.columns[22],df_norm_data.columns[23], df_norm_data.columns[24],
                              df_norm_data.columns[25], df_norm_data.columns[26]))
                             
feature_normalization= feature_normalization[0]


In [ ]:
def normalization(feature,norm_data):
    min_val = norm_data[feature].min() # record scaling minimum
    max_val = norm_data[feature].max() # record scaling maximum
    norm_data[feature] = (norm_data[feature] - min_val) / (max_val - min_val)
for i in range (len(feature_normalization)):
    normalization(feature_normalization[i],df_norm_data)

 ### Data Correlation

In [ ]:
# sns.set(font_scale=1.5)
# sns_plot = sns.pairplot(df_norm_data ,hue='class', palette='deep', vars=['ndvi', 'brightness_change_deviation','ratio_rg_change_deviation','brightness','relative_relief'],height = 4) #"b4", "b3", "b2",
# sns_plot.savefig(correlation_graph_output_path)


# 2. Classification

## Training and testing 

In [ ]:
import sklearn
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support as performance
from sklearn.metrics import classification_report,confusion_matrix
from plot_metric.functions import BinaryClassification
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from inspect import signature

### Define X (observations) and Y (predictions)

In [ ]:
y = df_keys ['class'] 

X = df_keys[['ndvi','ratio_rg_change_deviation','brightness_change_deviation','ndvi_change_deviation','brightness','slope_mean',
                'gndvi_deviation','slope_max','nd_std','relative_relief']]
# ,,,

# #creates arrays for X and y
y_array= y.values
X_array= X.values


In [ ]:
#get the name of the columns (to create the feature importance graph in step x)
feature_list = X.columns
feature_list

### Split the dataset 70%(training) 30%(testing)

In [ ]:
#Split the dataset in training and testing
def model_split(X,y):
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=28) 
    return X_train,X_test,y_train, y_test

### Functions to evaluate model performance

In [ ]:
#name of the classes: 1 = landslides; 0 =  non-landslides
class_name = df_keys['class'].unique()

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    
#    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           #label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='Actual class',
           xlabel='Predicted class')
#     plt.grid(b=None)
    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    
    #export the confusion matrix
    fig.savefig(confusion_matrix_output_path ) 
    
    return ax 
plt.show()


In [ ]:
def feature_importance(classifier,X_train):
    
     importances = list(classifier.feature_importances_)
    
    # List of tuples with variable and importance
     feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

    # Sort the feature importances by most important first
     feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

In [ ]:
def feature_importance_graph(classifier):
    
     # Get numerical feature importances
     importances = list(classifier.feature_importances_)
    
    # List of tuples with variable and importance
     feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

     # Sort the feature importances by most important first
     feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

     # list of x locations for plotting
     x_values = list(range(len(importances)))

    # Make a bar chart
     plt.bar(x_values, importances, orientation = 'vertical', linewidth = 0.9) #color = 'b', edgecolor = 'b'

     # Tick labels for x axis
     plt.xticks(x_values, feature_list, rotation='vertical')

    # Axis labels and title
     plt.ylabel('Importance'); plt.xlabel(''); plt.title('Feature Importances');    
        
     plt.savefig(feature_importance_output_path) 
    
    # List of features sorted from most to least important
     sorted_importances = [importance[1] for importance in feature_importances]
     sorted_features = [importance[0] for importance in feature_importances]
    
    # #     # Print out the feature and importances 
     [print('Variable: {:20}Importance: {}'.format(*pair)) for pair in feature_importances];    

In [ ]:

def model_performance(y_test,prediction_sampling):
    
        print( '\n'+'Classification_report:'+'\n'+'\n',classification_report(y_test,prediction_sampling))
        plot_confusion_matrix(y_test, prediction_sampling, classes=class_name,title='Confusion matrix')
        
        return classification_report(y_test,prediction_sampling, output_dict=True)

### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rforest = RandomForestClassifier(n_estimators=50, max_depth=40,bootstrap=True, class_weight={0:1,1:5},random_state=82,criterion="gini", min_samples_leaf=6, min_samples_split=4,max_features= 'auto')
rforest

In [ ]:
def train_model (model,X_train, y_train, X_test,y_test):
    
    model.fit (X_train, y_train)
    predictions = model.predict(X_test)
    metrics = model_performance(y_test,predictions)
    print ('------------------------------------------------------'+ '\n')
    
    return metrics

In [ ]:
def model_run(model, X,y): 
    
    X_train,X_test,y_train, y_test = model_split(X,y)

    train_model (model,X_train, y_train, X_test, y_test)
    
    feature_importance(model,X_train)

In [ ]:
model_run(rforest,X_array,y_array) #create the confusion matrix for the positive class

In [ ]:
feature_importance_graph(rforest) #ranking the features

# 3. Model Persistance

 Persist the model for future use without having to retrain. 

In [ ]:
y_model = df_keys ['class'] 

X_model = df_keys[['ndvi','ratio_rg_change_deviation','brightness_change_deviation','ndvi_change_deviation','brightness','slope_mean',
                'gndvi_deviation','slope_max','nd_std','relative_relief']]

#B8  #relativ

In [ ]:
y_array_= y_model.values
X_array_= X_model.values

In [ ]:
rf_classifier =   RandomForestClassifier(n_estimators=50, max_depth=40,bootstrap=True, class_weight={0:1,1:5},random_state=82,criterion="gini", min_samples_leaf=6, min_samples_split=4,max_features= 'auto')
rforest

In [ ]:
rf_classifier.fit (X_array_, y_array_)

Create model file


In [ ]:
from tempfile import mkdtemp
savedir = mkdtemp()
import os
from joblib import dump, load


In [ ]:
from joblib import dump, load
filename =  model_output_path

In [ ]:
dump(rf_classifier, filename) 

